# Fine-tune a YOLOv8 detection model

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

This notebook expects that the dataset created in the notebook [01_Dataset](./01_Dataset.ipynb) has been saved to disk.

### Setup

To start, check GPU support.

In [ ]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

### Load dataset

We'll first load the dataset created on disk.

In [ ]:
from pathlib import Path

dataset_dir = Path() / "dataset"

In [ ]:
import fiftyone as fo

# The splits to load
splits = ["train", "val", "test"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset()
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

In [ ]:
session = fo.launch_app(dataset, auto=False)

In [ ]:
session.show()

### Train a YoloV8 model

We can now train a YoloV8 base model on our exported dataset. We use the `yolov8n.pt` (nano) model, which is the smallest, but larger models are available from ultralytics. Simply replace `yolov8n.pt` with the [right model](https://docs.ultralytics.com/models/yolov8/#key-features).

In [ ]:
from adomvi.yolo.yolo import train

results_train = train("yolov8m.pt", data=dataset_dir / "dataset.yaml", batch=64, device=0)

#Load the path of the training model results
results_train_dir = Path(results_train.save_dir)


### Use the fine-tuned model to predict detections

Once our model is trained, we can simply use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [ ]:
from pathlib import Path
from adomvi.yolo.yolo import predict

best_model = Path() / results_train_dir / "weights/best.pt"
results_predict = predict(best_model, source=dataset_dir / "images/test")

# Load the path of the prediction model results
results_predict_dir = Path(results_predict[0].save_dir)

In [ ]:
import pickle

# Save the best_model variable to a file
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

### Evaluate model predictions

We can first evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We select the images tagged as part of the test set in our dataset.

In [ ]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [ ]:
from pathlib import Path
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / results_predict_dir / "labels"
add_yolo_detections(test_view, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["AFV", "APC", "MEV", "LAV"])

Now we can visualize these YOLOv8 model predictions on the samples in our dataset in the FiftyOne app:

In [ ]:
session.show()

Finally, we can evaluate our model's predictions and print the mAP.

In [ ]:
detection_results = test_view.evaluate_detections(
    prediction_field, 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

In [ ]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

In [ ]:
detection_results.print_report()